In [4]:
import folium
import pandas as pd
from tools import *

In [10]:
PER_INHABITANTS = 100000
scale = 0.15
YEAR = 2013
DISEASE = "RESPIRATORY"

In [5]:
NET_EMISSION_FILE_PATH = './/sdg_13_10_esmsip2.sdmx//UNFCCC_v26.csv'
MED_FILE_PATH = "./HFA_212_RESPIRATORY_EN.xlsx"
POPULATION_FILE_PATH = "./tps00001.tsv/tps00001.tsv"

In [6]:
grouped_df, europe, pop_df = get_world(POPULATION_FILE_PATH, NET_EMISSION_FILE_PATH)

100%|██████████| 53/53 [00:00<00:00, 2372.34it/s]


In [8]:
grouped_df

,name,Year,emissions
0,Austria,1985,-3384.71466
1,Austria,1986,-3428.59552
2,Austria,1987,-3067.21492
3,Austria,1988,-4935.99840
4,Austria,1989,-6134.86428
...,...,...,...
1879,Switzerland,2017,128828.38608
1880,Switzerland,2018,205013.61252
1881,Switzerland,2019,66707.06852
1882,Switzerland,2020,24427.80405


In [7]:
med_selected_countries = get_med_data(europe, MED_FILE_PATH)


In [11]:
selected_countries, min_value, max_value = compute_heatmap_per_year(YEAR, europe, grouped_df)
points_df, med_selected_countries = compute_points(pop_df, med_selected_countries, YEAR, PER_INHABITANTS, scale)

0 7201328.42322


100%|██████████| 30/30 [00:07<00:00,  4.04it/s]


In [13]:
points_df

,name,geometry,longitude,latitude
0,France,POINT (3.954758036677571 46.75288056061666),3.954758,46.752881
1,France,POINT (3.2656766163230664 43.460840481410784),3.265677,43.460840
2,France,POINT (-0.4558170550483984 45.94129335934726),-0.455817,45.941293
3,France,POINT (4.226029197388671 46.089313682341604),4.226029,46.089314
4,France,POINT (4.606473545760204 45.725505658455425),4.606474,45.725506
...,...,...,...,...
24639,Norway,POINT (10.54952217519665 63.78399752027214),10.549522,63.783998
24640,Norway,POINT (6.603579891192766 61.85630773183158),6.603580,61.856308
24641,Norway,POINT (6.997102204433734 60.12266783587172),6.997102,60.122668
24642,Norway,POINT (10.832492052132508 62.673908749181756),10.832492,62.673909


In [74]:
# Load built-in GeoJSON file containing country boundaries from geopandas datasets
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Calculate the centroid for each country
world['centroid'] = world.geometry.centroid

# Create a new GeoDataFrame with just the country name and the centroid
centroid_gdf = world[['name', 'geometry']]

,name,geometry
0,Fiji,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,Tanzania,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,W. Sahara,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,Canada,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,United States of America,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."
...,...,...
172,Serbia,"POLYGON ((18.82982 45.90887, 18.82984 45.90888..."
173,Montenegro,"POLYGON ((20.07070 42.58863, 19.80161 42.50009..."
174,Kosovo,"POLYGON ((20.59025 41.85541, 20.52295 42.21787..."
175,Trinidad and Tobago,"POLYGON ((-61.68000 10.76000, -61.10500 10.890..."


In [109]:
# Filter the data for a single year, say 2021
year_data = grouped_df[grouped_df['Year'] == 2021]
year_data = year_data.drop(columns=['Year'])
year_data = year_data.merge(centroid_gdf, left_on='name', right_on='name')

gdf = gpd.GeoDataFrame(year_data, geometry='geometry')

# Set CRS for the GeoDataFrame
gdf.crs = "EPSG:4326"

# Write the GeoDataFrame to a GeoJSON file
geojson_path = 'countries_emissions.geojson'
gdf.to_file(geojson_path, driver='GeoJSON')

In [110]:
grouped_df.columns

Index(['name', 'Year', 'emissions'], dtype='object')

In [106]:
# Initialize the map centered around Europe (Austria and Switzerland are in Europe)
m = folium.Map(location=[47.5162, 14.5501], zoom_start=4)  # Austria's approximate center

# Create a choropleth layer
choropleth = folium.Choropleth(
    geo_data=geojson_path,
    name='Emissions',
    data=year_data,
    columns=['name', 'emissions'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Emissions Intensity',
    highlight=True,
    nan_fill_color='black'  # or 'white' as per your comment
).add_to(m)

points_gdf = gpd.GeoDataFrame(points_df, geometry='geometry')
points_gdf.crs = gdf.crs
marker_group = folium.FeatureGroup(name='Respiratory Deaths scale: ' + str(scale))

# Add CircleMarker to the map
for idx, row in points_gdf.iterrows():
    folium.CircleMarker(
        location=[row['geometry'].y, row['geometry'].x],
        radius=0.01,  # Small radius for small points
        color='black',  # Orange color for the point
        fill=False,
        fill_opacity=0.0001,
        legend_name='Respiratory Deaths',
        popup=row['name']  # Popup text, can be changed to something else
    ).add_to(marker_group)

marker_group.add_to(m)

# Add layer control to toggle the choropleth on and off
folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('interactive_emissions_map.html')
